In [1]:
import music21 as ms
import numpy as np
import os

In [4]:
midi = {}
for file in os.listdir('data/'):
    try:
        haydn = ms.converter.parse('data/'+file)
    except:
        pass
    for nt in haydn[0]:
        if type(nt) == ms.tempo.MetronomeMark:
            midi[file] = nt.number
            break

In [6]:
data_files = []
for k in midi.keys():
    if midi[k] <= np.mean(list(midi.values()))+5 and midi[k] >= np.mean(list(midi.values()))-5:
        data_files.append('data/'+k)

In [7]:
from encoder_decoder import encode, decode

In [49]:
for file in data_files:
    try:
        music_file = ms.converter.parse(file)
        right_notes_encoded = encode(music_file[0],0.1)
        left_notes_encoded = encode(music_file[1],0.1)
        np.savetxt('data/right_'+file.split('/')[1].split('.')[0]+".csv", right_notes_encoded, delimiter=",")
        np.savetxt('data/left_'+file.split('/')[1].split('.')[0]+".csv", left_notes_encoded, delimiter=",")
    except:
        pass

In [50]:
# Prepare one master dataset
count = 1
files = os.listdir('data/')
files.sort()
for file in files:
    if file.split('_')[0] == 'left':
        if count == 1:
            if 'csv' in file:
                print('Processing ... '+file)
                temp = np.genfromtxt('data/'+file, delimiter=',')
        else:
            print('Processing ... '+file)
            csv = np.genfromtxt('data/'+file, delimiter=',')
            temp = np.concatenate(temp, csv)


Processing ... left_haydn_33_1.csv
Processing ... left_haydn_33_3.csv
Processing ... left_haydn_35_2.csv
Processing ... left_haydn_43_2.csv
Processing ... left_haydn_9_3.csv


In [51]:
left = temp.copy()
np.savetxt('left.csv', left, delimiter=",")

In [52]:
# Prepare one master dataset
count = 1
for file in files:
    if file.split('_')[0] == 'right':
        if count == 1:
            if 'csv' in file:
                print('Processing ... '+file)
                temp = np.genfromtxt('data/'+file, delimiter=',')
        else:
            print('Processing ... '+file)
            csv = np.genfromtxt('data/'+file, delimiter=',')
            temp = np.concatenate(temp, csv)


Processing ... right_haydn_33_1.csv
Processing ... right_haydn_33_3.csv
Processing ... right_haydn_35_2.csv
Processing ... right_haydn_43_2.csv
Processing ... right_haydn_9_3.csv


In [53]:
right = temp.copy()
np.savetxt('right.csv', right, delimiter=",")

In [54]:
np.mean(list(midi.values()))

118.74260869565217

In [55]:
new_right = decode(right, np.mean(list(midi.values())), 0.25)
new_right.write('midi','random_right.mid')

'random_right.mid'

In [56]:
new_left = decode(left, np.mean(list(midi.values())), 0.25)
new_left.write('midi','random_left.mid')

'random_left.mid'

In [57]:
sc = ms.stream.Stream()

In [58]:
sc.append(new_right)
sc.append(new_left)
new_left.offset=0.0
sc.write('midi','new.mid')


'new.mid'

In [59]:
data = np.genfromtxt('final.csv',delimiter=',')

In [61]:
midi = decode(data, np.mean(list(midi.values())), 0.10)

In [62]:
midi.write('midi','avi_test.mid')

'avi_test.mid'